# <span style="font-width:bold; font-size: 3rem; color:#1EB182;"><img src="../images/icon102.png" width="38px"></img> **Hopsworks Feature Store** </span><span style="font-width:bold; font-size: 3rem; color:#333;">- Part 03: Batch Inference</span>


## 🗒️ This notebook is divided into the following sections:

1. Load batch data.
2. Predict using model from Model Registry.

## <span style='color:#ff5f27'> 📝 Imports

In [ ]:
from xgboost import XGBClassifier

## <span style="color:#ff5f27;"> 📡 Connecting to Hopsworks Feature Store </span>

In [ ]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()

## <span style="color:#ff5f27;"> ⚙️ Feature View Retrieval</span>


In [ ]:
# Retrieve the 'transactions_view_fraud_batch_fv' feature view
feature_view = fs.get_feature_view(
    name='transactions_view_fraud_batch_fv',
    version=1,
)

## <span style="color:#ff5f27;">🗄 Model Registry</span>


In [ ]:
# Get the model registry
mr = project.get_model_registry()

## <span style='color:#ff5f27'>🚀 Fetch and test the model</span>

Finally you can start making predictions with your model! Retrieve your model from Hopsworks model registry.

In [ ]:
# Retrieve the model from the model registry
retrieved_model = mr.get_model(
    name="xgboost_fraud_batch_model",
    version=1,
)

# Download the saved model files to a local directory
saved_model_dir = retrieved_model.download()

In [ ]:
# Initialize the model
model = XGBClassifier()

# Load the model from a saved JSON file
model.load_model(saved_model_dir + "/model.json")
model

---
## <span style="color:#ff5f27;">🔮  Batch Prediction and Logging </span>


In [ ]:
# Initialize batch scoring
feature_view.init_batch_scoring(1)

# Get the untransformed and untransformed batch data for logging
untransformed_batch_data = feature_view.get_batch_data(transformed=False)
# transformed_batch_data = feature_view.transform(untransformed_batch_data)
transformed_batch_data = feature_view.get_batch_data()

# Drop the "datetime" column from the batch_data DataFrame along the specified axis (axis=1 means columns)
batch_data = transformed_batch_data.drop(["datetime"], axis=1)

# Display the first 3 rows
batch_data.head(3)

In [ ]:
# Use the retrieved XGBoost model to make predictions on the batch data
predictions = model.predict(batch_data)

# Display the first five predictions
predictions[:5]

In [ ]:
# log both transformed and untransformed features
feature_view.log(untransformed_batch_data.head(1000), predictions[:1000], training_dataset_version=1, model=retrieved_model)
feature_view.log(transformed_batch_data.head(1000), predictions[:1000], training_dataset_version=1, transformed=True, model=retrieved_model)

In [ ]:
# stop the job materialization schedule and materialize log manually
feature_view.pause_logging()
feature_view.materialize_log(wait=True)

In [ ]:
# read untransformed log
feature_view.read_log().head(3)

In [ ]:
# read transformed log
feature_view.read_log(transformed=True).head(3)

---

### <span style="color:#ff5f27;">🥳 <b> Next Steps  </b> </span>
Congratulations you've now completed the Fraud Batch tutorial for Managed Hopsworks.

Check out our other tutorials on ➡ https://github.com/logicalclocks/hopsworks-tutorials

Or documentation at ➡ https://docs.hopsworks.ai